<a href="https://colab.research.google.com/github/velizhask/PemrosesanBahasaAlami-nlp/blob/main/Representasi%20Teks2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Representasi Teks

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
import numpy as np
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer


In [ ]:
# Dataset sederhana
data = {
    'text': [
        "I love this product",
        "This is the best thing ever",
        "I hate this item",
        "Worst purchase I've ever made",
        "I'm so happy with my purchase",
        "This is terrible"
    ],
    'label': [1, 1, 0, 0, 1, 0]  # 1: positif, 0: negatif
}

# Membuat DataFrame
df = pd.DataFrame(data)

print(df)

**`Word embedding`** merupakan representasi yang dipelajari untuk teks di mana kata-kata yang memiliki makna sama memiliki representasi yang serupa. Metode Word embedding mempelajari representasi vektor bernilai riil untuk kosakata berukuran tetap yang telah ditentukan sebelumnya dari korpus teks.

In [ ]:
from gensim.models import Word2Vec

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenisasi
    tokens = word_tokenize(text.lower())
    # Menghapus tanda baca dan stopwords, serta lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word not in string.punctuation]
    return tokens

# Menerapkan preprocessing pada dataset
df['tokenized'] = df['text'].apply(preprocess_text)

# Membuat dan melatih model Word2Vec
model = Word2Vec(sentences=df['tokenized'], vector_size=50, window=3, min_count=1, workers=4)

# Fungsi untuk mengubah kalimat menjadi vektor rata-rata
def get_sentence_vector(sentence):
    vector = np.zeros(50)  # ukuran vektor
    count = 0
    for word in sentence:
        if word in model.wv.key_to_index:  # Mengecek apakah kata ada dalam model
            vector += model.wv[word]
            count += 1
    if count > 0:
        vector /= count
    return vector

# Membuat fitur vektor untuk setiap kalimat
X = np.array([get_sentence_vector(sentence) for sentence in df['tokenized']])
y = df['label'].values


worddata = pd.DataFrame(X)
worddata

**`Sentence Embedding`** Ini mirip dengan penyematan kata, satu-satunya perbedaan adalah sebagai pengganti kata, kalimat direpresentasikan sebagai vektor numerik dalam ruang dimensi tinggi. Tujuan penyematan kalimat adalah untuk menangkap makna dan hubungan semantik antara kata-kata dalam sebuah kalimat, serta konteks di mana kalimat tersebut digunakan.

In [ ]:
pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

# Preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenisasi
    tokens = word_tokenize(text.lower())
    # Menghapus tanda baca dan stopwords, serta lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(tokens)


# Menerapkan preprocessing pada dataset
df['processed_text'] = df['text'].apply(preprocess_text)

# Menginisialisasi model Sentence-BERT
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Mendapatkan embedding kalimat
X = model.encode(df['processed_text'].tolist())
y = df['label'].values

sentence_data = pd.DataFrame(X)
sentence_data

`Doc2Vec` digunakan untuk mendapatkan document embeddings, yaitu representasi vektor untuk dokumen atau teks panjang (bukan hanya kalimat pendek). Ini merupakan perpanjangan dari Word2Vec yang bertujuan untuk merepresentasikan seluruh dokumen atau paragraf sebagai sebuah vektor. TaggedDocument digunakan untuk melabeli setiap dokumen atau teks dalam koleksi, karena Doc2Vec belajar melalui dokumen yang sudah diberi tag

**`Document Embedding`** mengacu pada proses merepresentasikan seluruh dokumen, seperti paragraf, artikel, atau buku, sebagai vektor tunggal. Ini menangkap tidak hanya makna dan konteks kalimat individu tetapi juga hubungan dan koherensi antara kalimat dalam dokumen.

In [ ]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# Preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenisasi
    tokens = word_tokenize(text.lower())
    # Menghapus tanda baca dan stopwords, serta lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word not in string.punctuation]
    return tokens

# Menerapkan preprocessing pada dataset
df['tokenized'] = df['text'].apply(preprocess_text)

# Membuat tagged documents untuk Doc2Vec
tagged_data = [TaggedDocument(words=row['tokenized'], tags=[i]) for i, row in df.iterrows()]

# Membuat dan melatih model Doc2Vec
model = Doc2Vec(vector_size=50, window=3, min_count=1, workers=4, epochs=40)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# Mengubah dokumen menjadi vektor
X = np.array([model.infer_vector(doc.words) for doc in tagged_data])
y = df['label'].values

doc_data = pd.DataFrame(X)
doc_data

In [ ]:
print(tagged_data)

**SOAL PRAKTIKUM**



*   Terapkan N-Gram, Word Embedding, Sentence Embedding dan Document Embedding ke data yang diolah pada praktikum pertemuan 3

In [ ]:
!kaggle datasets download -d ucupsedaya/gojek-app-reviews-bahasa-indonesia #copy API command
!unzip gojek-app-reviews-bahasa-indonesia.zip

In [ ]:
import pandas as pd

df= pd.read_csv('/content/GojekAppReviewV4.0.0-V4.9.3_Cleaned.csv')
df.head()

In [ ]:
# Mengambil hanya kolom 'review' dan 'rating'
df = df[['content', 'score']]
df

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
pd.reset_option('display.max_colwidth')
df = df.dropna()
df = df.drop_duplicates()
df

In [ ]:
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
# Fungsi untuk menghapus tanda baca
def clean_text(text):
    text = text.lower()  # Konversi ke huruf kecil
    text = re.sub(r'[^\w\s]', '', text)  # Hapus tanda baca
    text = re.sub(r'\d+', '', text)  # Hapus angka
    text = re.sub(r'(.)\1{1,}', r'\1', text)  # Menghapus karakter berulang lebih dari 2 kali
    return text

df['content']=df['content'].apply(clean_text)
df.head()

In [ ]:
def tokenize_text(text):
    return word_tokenize(text)

# Terapkan fungsi tokenisasi pada setiap elemen di kolom 'content'
df['content'] = df['content'].apply(tokenize_text)
df

In [ ]:
!pip install sastrawi

In [ ]:
sw = stopwords.words('indonesian')
# Fungsi untuk menghapus stopwords dari token-token kata
def remove_stopwords(tokens):
    filtered_tokens = [word for word in tokens if word.lower() not in sw]
    return filtered_tokens

# Menghapus stopwords dari kolom 'Tokenized Text'
df['content'] = df['content'].apply(remove_stopwords)
df

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Membuat stemmer dari Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(words):
    return [stemmer.stem(word) for word in words]

In [ ]:
# Menerapkan fungsi stemming pada kolom 'content'
# Menerapkan stemming hanya pada 5 baris pertama
df.loc[:4, 'content'] = df.loc[:4, 'content'].apply(stemming)
df

In [ ]:
df['content'] = df['content'].apply(lambda x: ' '.join(x)) #membuat isi content menjadi kalimat kembali
df

In [ ]:
# Menerapkan preprocessing pada dataset
df['tokenized'] = df['content'].apply(preprocess_text)

# Membuat dan melatih model Word2Vec
model = Word2Vec(sentences=df['tokenized'], vector_size=50, window=3, min_count=1, workers=4)

# Fungsi untuk mengubah kalimat menjadi vektor rata-rata
def get_word_vector(word):
    vector = np.zeros(50)  # ukuran vektor
    count = 0
    for word in word:
        if word in model.wv.key_to_index:  # Mengecek apakah kata ada dalam model
            vector += model.wv[word]
            count += 1
    if count > 0:
        vector /= count
    return vector

# Membuat fitur vektor untuk setiap kalimat
X = np.array([get_word_vector(word) for word in df['tokenized']])
y = df['score'].values


worddata = pd.DataFrame(X)
worddata

`# Sentence Embedding `

In [ ]:
# # Mengambil stopwords
sw = stopwords.words('indonesian')
stemmer = PorterStemmer()

# Fungsi preprocessing teks
def preprocess_text(text):
    # Tokenisasi
    tokens = word_tokenize(text.lower())
    # Menghapus tanda baca dan stopwords, serta melakukan stemming
    tokens = [stemmer.stem(word) for word in tokens if word not in sw and word not in string.punctuation]
    return ' '.join(tokens)

# Menerapkan preprocessing pada dataset
df['processed_text'] = df['content'].apply(preprocess_text)

# Menginisialisasi model Sentence-BERT
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Mendapatkan embedding kalimat
X = model.encode(df['processed_text'].tolist())
y = df['score'].values

# Menyimpan embedding ke dalam DataFrame
sentence_data = pd.DataFrame(X.numpy())

# Menampilkan 50 baris pertama
sentence_data.head(50)

In [ ]:
!pip install -U tensorflow tensorflow-hub

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Mengambil stopwords bahasa Indonesia
sw = stopwords.words('indonesian')

# Fungsi preprocessing teks
def preprocess_text(text):
    # Tokenisasi
    tokens = word_tokenize(text.lower())
    # Menghapus tanda baca dan stopwords
    tokens = [word for word in tokens if word not in sw and word not in string.punctuation]
    return ' '.join(tokens)

# Menerapkan preprocessing pada dataset
df['processed_text'] = df['content'].apply(preprocess_text)

# Mengunduh Universal Sentence Encoder dari TensorFlow Hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Mendapatkan embedding kalimat
X = embed(df['processed_text'].tolist())

# Menyimpan embedding ke dalam DataFrame
sentence_data = pd.DataFrame(X.numpy())  # Convert tensor to numpy

# Menampilkan 50 baris pertama embedding
sentence_data


`# Document Embedding`

In [ ]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument


sw = stopwords.words('indonesian')
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenisasi
    tokens = word_tokenize(text.lower())
    # Menghapus tanda baca dan stopwords, serta lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in sw and word not in string.punctuation]
    return tokens

# Menerapkan preprocessing pada dataset
df['tokenized'] = df['content'].apply(preprocess_text)

# Membuat tagged documents untuk Doc2Vec
tagged_data = [TaggedDocument(words=row['tokenized'], tags=[i]) for i, row in df.iterrows()]

# Membuat dan melatih model Doc2Vec
model = Doc2Vec(vector_size=50, window=3, min_count=1, workers=4, epochs=40)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# Mengubah dokumen menjadi vektor
X = np.array([model.infer_vector(doc.words) for doc in tagged_data])
y = df['score'].values

doc_data = pd.DataFrame(X)
doc_data

Note:

Jika data kalian menggunakan bahasa inggris tidak perlu menggunakan module Sastrawi, ganti dengan PorterStemmer dan WordNetLemmatizer kemudian pada proses embbedding jangan lupa tambahkan:

Lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

menyesuaikan dengan baris program yang dibutuhkan